In [2]:
import numpy as np
from mesa import Agent, Model
from mesa.time import RandomActivation

c:\Users\leona\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\leona\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
class MoneyAgent(Agent):
    '''initial wealth'''

    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth  = 1

class MoneyModel(Model):
    '''model with some number of agents'''

    def __init__(self, N):
        self.num_agents = N
        # Create Agents
        for i in range(self.num_agents): 
            a = MoneyAgent(i, self)
            #print('Hi ! I am agent number ', i )
            self.schedule.add(a)

    def step(self):
        '''advanced the model by one step'''
        self.schedule.step()

In [5]:
x = MoneyModel(5)

AttributeError: 'MoneyModel' object has no attribute 'schedule'